In [1]:
# %%javascript
# IPython.OutputArea.auto_scroll_threshold = 9999;

In [2]:
import datetime
import os
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import os

from notebook_shared import decomposeutils as dcu
from notebook_shared import utils

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

from pandas import Series

import seaborn as sns
sns.set()
matplotlib.style.use('seaborn-colorblind')

INPUT_FILE = "dataset"
FSIZE = "full" #"tiny", "small" or "full"

df = pd.read_parquet(utils.get_dataset_path(INPUT_FILE, FSIZE))

In [3]:
from IPython.display import display

def output_decompose(provider, region, dt_rounding, decompose_result):
    display(decompose_result['decomposition']['fig'])
    display(decompose_result['seasonal_analysis']['fig'])
    print(decompose_result['stats_test']['case_desc'])
    print(decompose_result['stats_test']['adfuller']['result']['text'])
    print(decompose_result['stats_test']['kpss']['result']['text'])
    display(decompose_result['stats_test']['fig_corr'])
    display(decompose_result['stats_test']['fig_dist'])
    

In [4]:
def save_decompose(label, decompose_result):
    
    provider = decompose_result['parameters']['provider']
    region = decompose_result['parameters']['region']
    decompose_col = decompose_result['parameters']['decompose_col']
    
    if decompose_result['stats_test'] is None:
        utils.savefig(
            decompose_result['decomposition']['fig'],
            ['temporal_analysis', 'decomposition', decompose_col, provider, f'{region}_{label}']
        )
    else:            
        utils.savefig(
            decompose_result['decomposition']['fig'],
            ['temporal_analysis', 'decomposition', decompose_col, provider, f'{region}_{label}'],
            case=decompose_result['stats_test']['case_desc'],
            adfuller=decompose_result['stats_test']['adfuller']['result']['text'],
            kpss=decompose_result['stats_test']['kpss']['result']['text'])

        utils.savefig(
            decompose_result['stats_test']['fig_dist'],
            ['temporal_analysis', 'decomposition', decompose_col, provider, f'{region}_{label}_dist']
        )

        utils.savefig(
            decompose_result['stats_test']['fig_dist'],
            ['temporal_analysis', 'decomposition', decompose_col, provider, f'{region}_{label}_corr']
        )

    utils.savefig(
            decompose_result['seasonal_analysis']['fig'],
            ['temporal_analysis', 'decomposition', decompose_col, provider, f'{region}_{label}_seasonal']
        )

# Decomposition Runtime Half-Hourly, Hourly, Daily

In [5]:
decompositions_params = [
    {"label" : 'half-hourly', "dt_rounding" : '30min', "adflag" : 48, 'dtfmt':'%d/%m (%a)',  "providers" : ['AWS','AZURE', 'GCP'] },
    {"label" : 'hourly',      "dt_rounding" : 'H',     "adflag" : 24, 'dtfmt':'%d/%m (%a)',  "providers" : ['AWS','AZURE', 'GCP'] },
    {"label" : 'daily',       "dt_rounding" : 'D',     "adflag" : 7,  'dtfmt':'%d/%m (%a)',  "providers" : ['AWS','AZURE', 'GCP'] }
]

In [6]:
%matplotlib agg
from concurrent.futures import ThreadPoolExecutor 
import warnings
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter('ignore', InterpolationWarning)
import multiprocessing

decompose_runtime_results = {}
cpus = 4 # otherwise memory dies

sns.set(font_scale=1.5)
decomp_dataset = df
for decomp_param in decompositions_params:
    label, dt_rounding, adflag, dtfmt, providers = decomp_param.values()
    decompositions = []
    for provider in providers:
        regions = list(decomp_dataset[decomp_dataset['provider'] == provider].groupby(['provider'], observed=True)['region'].unique().values[0])
        
        def process(region):
            decomp = dcu.decompose(decomp_dataset, provider, region, 'runtime', dt_rounding, adflag=adflag, test_regression='ct', dtfmt=dtfmt)
            decompositions.append(decomp)
            return region
        
        with ThreadPoolExecutor(max_workers=cpus) as executor:
            results = list(executor.map(process, regions))
            
        executor.shutdown(wait=True)
        print('processed:', provider, results)
    decompose_runtime_results[label] = decompositions
%matplotlib inline

processed: AWS ['ap-northeast-1', 'ap-south-1', 'ap-southeast-2', 'ca-central-1', 'eu-central-1', 'eu-west-2', 'sa-east-1', 'us-east-1', 'us-west-1', 'us-west-2']
processed: AZURE ['australiaeast', 'brazilsouth', 'centralindia', 'eastus', 'germanywestcentral', 'japaneast', 'uksouth', 'westus', 'westus2']
processed: GCP ['asia-northeast1', 'asia-south1', 'australia-southeast1', 'europe-west2', 'europe-west3', 'northamerica-northeast1', 'southamerica-east1', 'us-east4', 'us-west2', 'us-west4']
processed: AWS ['ap-northeast-1', 'ap-south-1', 'ap-southeast-2', 'ca-central-1', 'eu-central-1', 'eu-west-2', 'sa-east-1', 'us-east-1', 'us-west-1', 'us-west-2']
processed: AZURE ['australiaeast', 'brazilsouth', 'centralindia', 'eastus', 'germanywestcentral', 'japaneast', 'uksouth', 'westus', 'westus2']
processed: GCP ['asia-northeast1', 'asia-south1', 'australia-southeast1', 'europe-west2', 'europe-west3', 'northamerica-northeast1', 'southamerica-east1', 'us-east4', 'us-west2', 'us-west4']
proces

In [7]:
fdir = os.path.join('pickles', 'temporal_analysis')
os.makedirs(fdir, exist_ok=True)

labels = ['half-hourly', 'hourly', 'daily']
for label in labels:
    for result in decompose_runtime_results[label]:
        decompose_col = result['parameters']['decompose_col']
        # output_decompose(provider, region, label, result)
        save_decompose(label, result)
        with open(os.path.join(fdir, f'decomposition-{decompose_col}-{FSIZE}-{label}.p'), 'wb') as f:
            pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

# Decomposition No CPU per Invocation Half-Hourly, Hourly, Daily

In [8]:
# Create CPU Counts
df_cpus = df[['provider', 'region', 'timezone', 'cpuType', 'workload_invocation', 'driver_invocation']]
df_cpu_counts = df_cpus.groupby(['driver_invocation','provider', 'region', 'timezone', 'workload_invocation'], observed=True)['cpuType'].nunique().rename('NoCpuTypeInvocation')
df_cpu_counts = pd.DataFrame(df_cpu_counts).reset_index()
df_cpu_counts.groupby(['provider', 'region'], observed=True).describe()

NoCpuTypeInvocation                           \
                                               count      mean       std  min   
provider region                                                                 
AWS      ap-northeast-1                       2895.0  1.014508  0.119592  1.0   
         ap-south-1                           2894.0  1.045957  0.209428  1.0   
         ap-southeast-2                       2896.0  1.034530  0.182619  1.0   
         ca-central-1                         2897.0  1.072144  0.258770  1.0   
         eu-central-1                         2895.0  1.011744  0.107752  1.0   
         eu-west-2                            2893.0  1.002765  0.052522  1.0   
         sa-east-1                            2894.0  1.019696  0.138977  1.0   
         us-east-1                            2887.0  1.200208  0.400225  1.0   
         us-west-1                            2893.0  1.092637  0.289974  1.0   
         us-west-2                            2886.0  1.001733  0.041594  1.0   
AZURE    australiaeast                        2884.0  1.901179  0.664974  1.0   
         brazilsouth                          2880.0  2.062500  0.642576  1.0   
         centralindia                         2895.0  1.450086  0.502426  1.0   
         eastus                               2878.0  2.230368  0.585500  1.0   
         germanywestcentral                   2898.0  1.651484  0.522202  1.0   
         japaneast                            2896.0  1.991713  0.619153  1.0   
         uksouth                              2897.0  2.031757  0.667176  1.0   
         westus                               2843.0  2.074217  0.609374  1.0   
         westus2                              2896.0  1.254144  0.497646  1.0   
GCP      asia-northeast1                      2900.0  2.682759  0.496326  1.0   
         asia-south1                          2901.0  1.000000  0.000000  1.0   
         australia-southeast1                 2901.0  1.971044  0.167710  1.0   
         europe-west2                         2901.0  1.255084  0.435984  1.0   
         europe-west3                         2901.0  1.925198  0.263117  1.0   
         northamerica-northeast1              2901.0  1.000000  0.000000  1.0   
         southamerica-east1                   2900.0  1.986207  0.116651  1.0   
         us-east4                             2901.0  1.895898  0.305446  1.0   
         us-west2                             2900.0  1.000000  0.000000  1.0   
         us-west4                             2900.0  1.000000  0.000000  1.0   

                                                      
                                  25%  50%  75%  max  
provider region                                       
AWS      ap-northeast-1           1.0  1.0  1.0  2.0  
         ap-south-1               1.0  1.0  1.0  2.0  
         ap-southeast-2           1.0  1.0  1.0  2.0  
         ca-central-1             1.0  1.0  1.0  2.0  
         eu-central-1             1.0  1.0  1.0  2.0  
         eu-west-2                1.0  1.0  1.0  2.0  
         sa-east-1                1.0  1.0  1.0  2.0  
         us-east-1                1.0  1.0  1.0  2.0  
         us-west-1                1.0  1.0  1.0  2.0  
         us-west-2                1.0  1.0  1.0  2.0  
AZURE    australiaeast            1.0  2.0  2.0  4.0  
         brazilsouth              2.0  2.0  2.0  4.0  
         centralindia             1.0  1.0  2.0  3.0  
         eastus                   2.0  2.0  3.0  3.0  
         germanywestcentral       1.0  2.0  2.0  3.0  
         japaneast                2.0  2.0  2.0  4.0  
         uksouth                  2.0  2.0  2.0  4.0  
         westus                   2.0  2.0  2.0  3.0  
         westus2                  1.0  1.0  1.0  3.0  
GCP      asia-northeast1          2.0  3.0  3.0  3.0  
         asia-south1              1.0  1.0  1.0  1.0  
         australia-southeast1     2.0  2.0  2.0  2.0  
         europe-west2             1.0  1.0  2.0  2.0  
         e

In [9]:
%matplotlib agg
from concurrent.futures import ThreadPoolExecutor 
import warnings
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter('ignore', InterpolationWarning)
import multiprocessing

decompose_runtime_results = {}
cpus = 4 # otherwise memory dies
sns.set(font_scale=1.5)

decomp_dataset = df_cpu_counts
for decomp_param in decompositions_params:
    label, dt_rounding, adflag, dtfmt, providers = decomp_param.values()
    decompositions = []
    for provider in providers:
        regions = list(decomp_dataset[decomp_dataset['provider'] == provider].groupby(['provider'], observed=True)['region'].unique().values[0])
        
        def process(region):
            decomp = dcu.decompose(decomp_dataset, provider, region, 'NoCpuTypeInvocation', dt_rounding, 
                                   adflag=adflag, test_regression='ct', dtfmt=dtfmt,
                                   statsTest = False, agg='mean')
            decompositions.append(decomp)
            return region
        
        with ThreadPoolExecutor(max_workers=cpus) as executor:
            results = list(executor.map(process, regions))
            
        executor.shutdown(wait=True)
        print('processed:', provider, results)
    decompose_runtime_results[label] = decompositions
%matplotlib inline

processed: AWS ['ap-northeast-1', 'ap-south-1', 'ap-southeast-2', 'ca-central-1', 'eu-central-1', 'eu-west-2', 'sa-east-1', 'us-east-1', 'us-west-1', 'us-west-2']
processed: AZURE ['australiaeast', 'brazilsouth', 'centralindia', 'eastus', 'germanywestcentral', 'japaneast', 'uksouth', 'westus', 'westus2']
processed: GCP ['asia-northeast1', 'asia-south1', 'australia-southeast1', 'europe-west2', 'europe-west3', 'northamerica-northeast1', 'southamerica-east1', 'us-east4', 'us-west2', 'us-west4']
processed: AWS ['ap-northeast-1', 'ap-south-1', 'ap-southeast-2', 'ca-central-1', 'eu-central-1', 'eu-west-2', 'sa-east-1', 'us-east-1', 'us-west-1', 'us-west-2']
processed: AZURE ['australiaeast', 'brazilsouth', 'centralindia', 'eastus', 'germanywestcentral', 'japaneast', 'uksouth', 'westus', 'westus2']
processed: GCP ['asia-northeast1', 'asia-south1', 'australia-southeast1', 'europe-west2', 'europe-west3', 'northamerica-northeast1', 'southamerica-east1', 'us-east4', 'us-west2', 'us-west4']
proces

In [10]:
fdir = os.path.join('pickles', 'temporal_analysis')
os.makedirs(fdir, exist_ok=True)

labels = ['half-hourly', 'hourly', 'daily']
for label in labels:
    for result in decompose_runtime_results[label]:
        decompose_col = result['parameters']['decompose_col']
        # output_decompose(provider, region, label, result)
        save_decompose(label, result)
        with open(os.path.join(fdir, f'decomposition-{decompose_col}-{FSIZE}-{label}.p'), 'wb') as f:
            pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)